<a href="https://colab.research.google.com/github/amine406/SeoProject/blob/main/Crawling_Multi_thread%C3%A9_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Initialiser les variables
links = set()  # Pour stocker les URLs visitées
page_info = []  # Stocker les infos SEO pour chaque URL

# Créer une session
session = requests.Session()

# Ajouter un User-Agent à la session
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
})

# Fonction pour vérifier si l'URL est valide et interne
def is_valid_url(_url, base_url):
    parsed_url = urlparse(_url)
    return bool(parsed_url.scheme) and bool(parsed_url.netloc) and base_url in _url

# Fonction pour crawler et analyser une page
def crawl_page(_url, base_url):
    try:
        response = session.get(_url, timeout=5)  # Timeout après 5 secondes
        http_status = response.status_code
        print(f"Crawling {_url} {http_status}")

        if http_status != 200:
            return []  # Ignorer si le status n'est pas OK

# Vérifier que la réponse est bien du HTML
        content_type = response.headers.get("Content-Type", "")
        print(f"Content-Type de {_url}: {content_type}")
        if "text/html" not in content_type:
            return []  # Ignorer le contenu non-HTML

# Utiliser un parser
        soup = BeautifulSoup(response.content, "html.parser")

# Récupérer les informations SEO
        title = soup.title.string if soup.title else "No title"
        description = soup.find("meta", attrs={"name": "description"})
        description = description["content"] if description else "No description"
        h1 = soup.find("h1")
        h1_text = h1.get_text() if h1 else "No H1"

# Compter le nombre de liens sur la page
        all_links = soup.find_all("a", href=True)
        link_count = len(all_links)
        print(f"{link_count} liens trouvés sur {_url}")

# Stocker les informations dans une liste
        page_info.append({
            "URL": _url,
            "HTTP Status": http_status,
            "Title": title,
            "Meta Description": description,
            "H1": h1_text,
            "Number of Links": link_count
        })

# Retourner la liste des liens internes
        return [urljoin(base_url, link.get('href')) for link in all_links if is_valid_url(urljoin(base_url, link.get('href')), base_url)]

    except requests.RequestException as e:
        print(f"Failed to crawl {_url}: {e}")
        return []  # Return an empty list if there is an error

# Fonction pour démarrer le crawling
def crawl_website(start_url, max_workers=5):
    base_url = "{0.scheme}://{0.netloc}".format(urlparse(start_url))
    pending_urls = [start_url]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        crawl = []

        while pending_urls:
            _url = pending_urls.pop()

            if _url not in links:
                links.add(_url)
                crawl.append(executor.submit(crawl_page, _url, base_url))

        for future in crawl:
            inew_links = future.result()

# sortir nouveaux liens trouvés
        print("Nouveaux liens trouvés :")
        for link in inew_links:
            print(link)

    pending_urls.extend(inew_links)

# Convertir les données en DataFrame
    df = pd.DataFrame(page_info)

    print("\nTableau récapitulatif des pages crawlées:")
    print(df)

# Exporter vers un fichier CSV
    if not df.empty:
        df.to_csv("seo_crawl_results_file.csv", index=False)
        print("Résultats exportés vers 'seo_crawl_results_file'.")
    else:
        print("Aucune donnée à exporter.")

# Démarrer le crawler sur le site
if __name__ == "__main__":
    print("Démarrage du crawling sur https://aundetailpres.fr...")
    crawl_website("https://aundetailpres.fr", max_workers=10)
    print("Crawling terminé.")


Démarrage du crawling sur https://aundetailpres.fr...
Crawling https://aundetailpres.fr 200
Content-Type de https://aundetailpres.fr: text/html; charset=UTF-8
289 liens trouvés sur https://aundetailpres.fr
Nouveaux liens trouvés :
https://aundetailpres.fr#content
https://aundetailpres.fr/
https://aundetailpres.fr/category/mariage-2/
https://aundetailpres.fr/category/mariage-2/decoration-et-sortie-deglise/
https://aundetailpres.fr/category/mariage-2/astuces-organisation-et-animation/
https://aundetailpres.fr/category/mariage-2/toute-la-decoration/
https://aundetailpres.fr/category/mariage-2/faire-part-mariage-2/
https://aundetailpres.fr/category/mariage-2/marque-place-menus-et-plan-de-table/
https://aundetailpres.fr/category/mariage-2/livre-dor-et-urne/
https://aundetailpres.fr/category/mariage-2/livret-de-messe-mariage-2/
https://aundetailpres.fr/category/mariage-2/textes-pour-la-ceremonie/
https://aundetailpres.fr/category/mariage-2/tout-pour-la-mariee/
https://aundetailpres.fr/catego